### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.3.0'

### Data Preprocessing

##### Importing the dataset

In [3]:
df = pd.read_csv('Churn_Modelling.csv')
X = df.iloc[:, 3:-1]
y = df.iloc[:, -1]

### Label Encoding Ordinal features or features with only 2 levels

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['Gender'] = le.fit_transform(X["Gender"])

### One Hot Encoding the Nominal features

In [5]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
# X = np.array(ct.fit_transform(X))

dumm = pd.get_dummies(X['Geography'])



### If get_dummies method is used, we have to concatenate the actual dataframe and the dataframe with the dummies of the categorical features.

### If OneHotEncoder was used, we need not concatenate as it will modify the actual dataframe.

In [7]:
X = pd.concat([X,dumm],axis = 1)

In [8]:
X = X.drop("Geography",1)

### Splitting the data into Train and Test

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### As Neural networks work on the basis of the product of the weights and the actual wegights, it is absolutely mandatory to scale the data.

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### ANN

##### Initializing ANN with input parameters

In [12]:
n = int(input("Enter number of layers: "))
neuron = int(input("Enter the number of neurons per layer: "))

ann = tf.keras.models.Sequential()
for i in range(1,n+1):
  ann.add(tf.keras.layers.Dense(units = neuron, activation='relu'))

ann.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Enter number of layers: 5
Enter the number of neurons per layer: 6


### Defining number of epochs and training the data

In [13]:
epoch = int(input("Enter the number of epochs: "))
ann.fit(X_train,y_train, batch_size=32, epochs = epoch)

Enter the number of epochs: 300
Epoch 1/300
250/250 [==============================] - 0s 984us/step - loss: 0.4985 - accuracy: 0.7945
Epoch 2/300
250/250 [==============================] - 0s 1ms/step - loss: 0.4552 - accuracy: 0.7945
Epoch 3/300
250/250 [==============================] - 0s 1ms/step - loss: 0.4383 - accuracy: 0.8001
Epoch 4/300
250/250 [==============================] - 0s 1ms/step - loss: 0.4323 - accuracy: 0.8079
Epoch 5/300
250/250 [==============================] - 0s 987us/step - loss: 0.4277 - accuracy: 0.8129
Epoch 6/300
250/250 [==============================] - 0s 1ms/step - loss: 0.4255 - accuracy: 0.8161
Epoch 7/300
250/250 [==============================] - 0s 1ms/step - loss: 0.4225 - accuracy: 0.8189
Epoch 8/300
250/250 [==============================] - 0s 1ms/step - loss: 0.4208 - accuracy: 0.8217
Epoch 9/300
250/250 [==============================] - 0s 963us/step - loss: 0.4185 - accuracy: 0.8248
Epoch 10/300
250/250 [==============================]

### <u>Checking the performance by comparing the predicted and actual results</u>

In [14]:
y_pred = ann.predict(X_test)

In [15]:
y_test = np.array(y_test).reshape(-1,1)

In [16]:
y_pred[:].shape

(2000, 1)

##### As the resultant values are between 0 and 1 (due to the sigmoid activation function), we have to define a threshold to convert a value to either 0 or 1.

In [20]:
from sklearn.preprocessing import binarize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,f1_score
for i in range(1,11):
    y_pred2=binarize(y_pred,i/10)[:]
    cm2=confusion_matrix(y_test,y_pred2)
    print ('With',i/10,'threshold the Confusion Matrix is ','\n',cm2,'\n',
            'with',cm2[0,0]+cm2[1,1],'correct predictions and',cm2[1,0],'Type II errors( False Negatives)','\n\n',
          'Sensitivity: ',cm2[1,1]/(float(cm2[1,1]+cm2[1,0])),'Specificity: ',cm2[0,0]/(float(cm2[0,0]+cm2[0,1])),'\n\n\n')
    print('The accuracy score is: ',accuracy_score(y_test,y_pred2))
    print('The f1 score is: ',f1_score(y_test,y_pred2))
    print('\n')

With 0.1 threshold the Confusion Matrix is  
 [[881 726]
 [ 41 352]] 
 with 1233 correct predictions and 41 Type II errors( False Negatives) 

 Sensitivity:  0.8956743002544529 Specificity:  0.5482265090230243 



The accuracy score is:  0.6165
The f1 score is:  0.4785859959211421


With 0.2 threshold the Confusion Matrix is  
 [[1220  387]
 [  82  311]] 
 with 1531 correct predictions and 82 Type II errors( False Negatives) 

 Sensitivity:  0.7913486005089059 Specificity:  0.7591785936527692 



The accuracy score is:  0.7655
The f1 score is:  0.5701191567369387


With 0.3 threshold the Confusion Matrix is  
 [[1405  202]
 [ 132  261]] 
 with 1666 correct predictions and 132 Type II errors( False Negatives) 

 Sensitivity:  0.6641221374045801 Specificity:  0.8742999377722465 



The accuracy score is:  0.833
The f1 score is:  0.6098130841121495


With 0.4 threshold the Confusion Matrix is  
 [[1493  114]
 [ 189  204]] 
 with 1697 correct predictions and 189 Type II errors( False Negat

Threshold of 0.5 yields the best result. Hence,
- [values >= 0.5] = 1 
- [values < 0.5] = 0

The resultant values can be used as the predicted output from the model.

In [35]:
y_pred2=binarize(y_pred,0.5)[:]

In [26]:
df.head(1)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [24]:
X_test[0]

array([-0.57749609,  0.91324755, -0.6557859 , -0.69539349,  0.32993735,
        0.80843615, -1.54035103, -1.02583358, -1.01960511, -0.99850112,
        1.72572313, -0.57638802])

### Checking with the customer details, whether the customer will churn or not!

In [36]:
result = ann.predict(sc.transform([[600,0,40,3,60000,2,1,1,50000,1,0,0]]))

In [37]:
result_=binarize(result,0.5)[:]

In [38]:
result_

array([[0.]], dtype=float32)

In [34]:
print("As the value of the prediction is ",int(result_[0])," we can retain the customer as he is not likely to switch")

As the value of the prediction is  0  we can retain the customer as he is not likely to switch


#### The number of layers and the number of neuron per layer depends on the problem and in this case, utilizing more number of Layers resulted in the model Over fitting to the Training data.